In [ ]:
import os
os.makedirs("../fig", exist_ok=True)


In [1]:
import numpy as np
rng = np.random.default_rng(0)


In [2]:
# Helper: generate random pointer tables T_j : [m] -> [m] for j=1..k

def generate_tables(m: int, k: int, rng: np.random.Generator):
    # Each T_j is represented as a length-m array with entries in [0, m-1]
    return [rng.integers(low=0, high=m, size=m, dtype=np.int64) for _ in range(k)]


def compose_pointer(T_list, s: int):
    u = s
    for T in T_list:
        u = int(T[u])
    return u


def ceil_log2(m: int) -> int:
    return int(np.ceil(np.log2(max(2, m))))


In [3]:
# Parameters and empirical estimation of fooling family size
m_values = [8, 16, 32, 64, 128]
k = 5  # depth parameter (k >= 2)
c = 1.0  # budget constant
alpha_target = 0.9

results = []
for m in m_values:
    # Fix T1..T_{k-1}, vary Tk and b on a large subset S (|S| >= 0.9 m)
    T_prefix = generate_tables(m, k-1, rng)
    s = int(rng.integers(low=0, high=m))
    # Construct S of size floor(0.9 m)
    S_size = int(np.floor(0.9 * m))
    alpha_emp = S_size / m
    # Encoding size: n = k*m*ceil(log2 m) + m
    n = k * m * ceil_log2(m) + m
    # Empirical proxy for log2 M using last-layer degrees of freedom over S
    # M ≈ 2^{alpha*m} => log2 M ≈ alpha*m
    log2_M_est = alpha_emp * m
    # Lower bound proxy: T ≥ log M / B(k-1,n) with B = c*(k-1)*log2(n)
    denom = c * (k - 1) * np.log2(max(2, n))
    T_lb_est = log2_M_est / denom
    results.append({
        'm': m,
        'n': n,
        'alpha_emp': alpha_emp,
        'log2_M_est': log2_M_est,
        'T_lb_est': T_lb_est
    })

# Display a compact table
print("m\talpha_emp\tlog2 M est (bits)\tT_lb_est")
for r in results:
    print(f"{r['m']}\t{r['alpha_emp']:.3f}\t{r['log2_M_est']:.2f}\t{r['T_lb_est']:.4f}")


m	alpha_emp	log2 M est (bits)	T_lb_est
8	0.875	7.00	0.2500
16	0.875	14.00	0.4170
32	0.875	28.00	0.7216
64	0.891	57.00	1.3009
128	0.898	115.00	2.3624


In [4]:
"""
We fix $T_1,\ldots,T_{k-1}$ and vary the last layer $(T_k, b)$ over a large subset $S \subseteq [m]$ with $|S| \ge 0.9m$.
As a proxy, we take $\log_2 M \approx \alpha m$ with $\alpha = |S|/m$.
The theoretical lower bound is $T \gtrsim (\log M) / (c (k-1) \log n)$.
"""


<>:2: SyntaxWarning: invalid escape sequence '\l'
<>:2: SyntaxWarning: invalid escape sequence '\l'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_40480\3232417594.py:2: SyntaxWarning: invalid escape sequence '\l'
  We fix $T_1,\ldots,T_{k-1}$ and vary the last layer $(T_k, b)$ over a large subset $S \subseteq [m]$ with $|S| \ge 0.9m$.


'\nWe fix $T_1,\\ldots,T_{k-1}$ and vary the last layer $(T_k, b)$ over a large subset $S \\subseteq [m]$ with $|S| \\ge 0.9m$.\nAs a proxy, we take $\\log_2 M \x07pprox \x07lpha m$ with $\x07lpha = |S|/m$.\nThe theoretical lower bound is $T \\gtrsim (\\log M) / (c (k-1) \\log n)$.\n'

In [ ]:
# ---- FIX: белый экран / не сохраняется ----
import os, sys
import matplotlib
matplotlib.use("Agg")  # гарантированно неинтерактивный backend
import matplotlib.pyplot as plt

# 1) чистим всё старое
plt.close("all")

# 2) убеждаемся, что папка для файлов существует
out_dir = "../fig"
os.makedirs(out_dir, exist_ok=True)

# 3) создаём фигуру и оси с белым фоном
fig, ax = plt.subplots(figsize=(10, 6), constrained_layout=True)
fig.patch.set_alpha(1.0)
fig.patch.set_facecolor("white")
ax.set_facecolor("white")

# === YOUR PLOT ===
# Адаптация исходного кода построения
ms = np.array([r['m'] for r in results], dtype=float)
log2M = np.array([r['log2_M_est'] for r in results], dtype=float)
alpha_emp = np.array([r['alpha_emp'] for r in results], dtype=float)

alpha_line = float(alpha_target)

ax.plot(ms, log2M, 'o-', label='empirical log2 M ≈ α·m')
ax.plot(ms, alpha_line * ms, '--', label=f'theoretical α·m (α={alpha_line:.2f})')
ax.set_xlabel('m')
ax.set_ylabel('α·m (theoretical)')
ax.set_title('Pointer-Chase L_k: α·m vs m')
ax.grid(True, alpha=0.3)
ax.legend()

# Report alpha and entropy comparison vs budget
for r in results:
    n = r['n']
    m = r['m']
    log2M_emp = r['log2_M_est']
    denom = c * (k - 1) * np.log2(max(2, n))
    print(f"m={m}, alpha_emp={r['alpha_emp']:.3f}, alpha_line={alpha_line:.2f}, T_lb_est={log2M_emp/denom:.4f}")
# === /YOUR PLOT ===

# 4) проверка, что что-то реально нарисовано
has_artists = (len(ax.lines) + len(ax.patches) + len(ax.collections) + len(ax.images)) > 0
if not has_artists:
    raise RuntimeError("На оси ничего не нарисовано. Добавь хотя бы ax.plot(...)/ax.scatter(...).")

# 5) финальный рендер + сохранение (непрозрачный белый фон, «поджать» поля)
out_path = os.path.join(out_dir, "lk_logM.png")
fig.canvas.draw()  # форсируем отрисовку
fig.savefig(
    out_path,
    dpi=300,
    bbox_inches="tight",
    facecolor=fig.get_facecolor(),
    edgecolor="none",
)

# 6) контроль веса файла
size_kb = os.path.getsize(out_path) / 1024
print(f"Saved to: {out_path} ({size_kb:.1f} KB)")


m=8, alpha_emp=0.875, alpha_line=0.90, T_lb_est=0.2500
m=16, alpha_emp=0.875, alpha_line=0.90, T_lb_est=0.4170
m=32, alpha_emp=0.875, alpha_line=0.90, T_lb_est=0.7216
m=64, alpha_emp=0.891, alpha_line=0.90, T_lb_est=1.3009
m=128, alpha_emp=0.898, alpha_line=0.90, T_lb_est=2.3624
Saved to: ../paper/fig\lk_logM.png (162.3 KB)


In [6]:
# Entropy analysis: actual vs theoretical α·m bound
for r in results:
    alpha_emp = r['alpha_emp']
    m = r['m']
    log2M_emp = r['log2_M_est']
    log2M_theory = alpha_target * m
    print(f"m={m}: alpha_emp={alpha_emp:.3f}, log2M_emp={log2M_emp:.2f}, theory={log2M_theory:.2f}")


m=8: alpha_emp=0.875, log2M_emp=7.00, theory=7.20
m=16: alpha_emp=0.875, log2M_emp=14.00, theory=14.40
m=32: alpha_emp=0.875, log2M_emp=28.00, theory=28.80
m=64: alpha_emp=0.891, log2M_emp=57.00, theory=57.60
m=128: alpha_emp=0.898, log2M_emp=115.00, theory=115.20
